In [5]:
from os import times
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from setuptools.command.install import install
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from trio import sleep
import requests
import base64
import pandas as pd
from urllib3 import request

import json
import os


In [6]:
def openDriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # DRIVER_PATH = "chromedriver.exe"
    return webdriver.Chrome(options=chrome_options)


In [7]:
def getLink(uri):
    driver = openDriver()
    driver.get(uri)
    time.sleep(1)
    
    links = []
    parent = driver.find_elements(By.CLASS_NAME, "plp-fragment-wrapper")
    for index in parent:
        root = ((index.find_element(By.CLASS_NAME, "plp-mastercard ").
                 find_element(By.TAG_NAME, "a")).
                get_attribute("href"))
        links.append(root)
        
    driver.quit()
    return links


In [8]:
def saveDataToJSON(directory, filename, data):
    os.makedirs("json/link/" + directory, exist_ok=True)
    try:
        with open(f'json/link/{directory}/{filename}.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)
            
    except FileNotFoundError:
        existingData = []
    
    existingData.extend(data)
    with open(f'json/link/{directory}/{filename}.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
        

In [9]:
# for i in range(12,14):
#     url = {
#         "furniture sets":
#                f"https://www.ikea.com/th/th/cat/furniture-sets-55036/?page={i}"
#     }
#     LinkList = getLink(url.get('furniture sets'))
#     saveDataToJSON("furniture sets","FurnitureSetsLink",LinkList)
#     print("page = ",i)
#     

In [10]:
def getLinkProduct(n, nameCat, urlCat):
    for i in range(1, n):
        uri = {
            nameCat:
                f"{urlCat}?page={i}"
        }
        linkList = getLink(uri.get(nameCat))
        saveDataToJSON(nameCat, f"{nameCat}Link", linkList)
        print("page = ", i)

In [11]:
def getDetailProduct(urlCat):
    driver = openDriver()
    driver.get(urlCat)
    
    allDetail = driver.find_element(By.XPATH, '//div[@class="js-price-package pip-price-package"]')
    brandName = allDetail.text.split("\n")[0]
    detail = allDetail.text.split("\n")[1].split(", ")
    productName = detail[0]
    try:
        color = detail[1]
    except:
        color = None
    try:
        size = detail[2]
    except:
        size = None
    price = allDetail.text.split("\n")[2][:-3]
    
    driver.quit()
    return brandName, productName,color, size, price

###กดปุ่ม เพื่อรับข้อมูล

In [12]:
def getDescFromProduct(uri):
    browser = openDriver()
    browser.get(uri)
    try:
        button = browser.find_element(By.CLASS_NAME, "pip-list-view-item__action")
        browser.execute_script("arguments[0].click();", button)
        
        paragraphs = browser.find_elements(By.CLASS_NAME, 'pip-product-details__paragraph')
        codeProduct = (browser.find_element(By.CLASS_NAME, 'pip-product-identifier__value').
                       get_attribute("innerText"))
        
        stringDesc = ""
        for paragraph in paragraphs:
            stringDesc += paragraph.get_attribute("innerText")+"\n\n"

        browser.quit()
        return stringDesc , codeProduct
        
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        browser.quit()

------------------------

In [13]:
# Download Image for Product
def saveImage(uri, dirName, fileName):
    response = requests.get(uri)
    
    if response.status_code == 200:
        os.makedirs("image/room/"+dirName, exist_ok=True)
        with open(f"image/room/{dirName}/{fileName}_image.jpg", "wb") as file:
            file.write(response.content)
    else:
        print("เกิดข้อผิดพลาดในการดาวน์โหลดภาพ:", response.status_code)


In [14]:
def kernelImage(dirName, fileName):
    with open(f"image/room/{dirName}/{fileName}_image.jpg", "rb") as png_file:
        # อ่านไฟล์ PNG
        png_data = png_file.read()
        # แปลงข้อมูลไฟล์ PNG เป็น Base64
        base64_encoded_data = base64.b64encode(png_data)
        # แปลง Base64 เป็น string เพื่อใช้งาน
        base64_string = base64_encoded_data.decode('utf-8')
        
    return base64_string
    
    

In [15]:
def getImage(uri, dirName, fileName):
    driver = openDriver()
    driver.get(uri)
    findElement = driver.find_element(By.XPATH, "//div[@class='pip-zoom-image']")
    img_element = findElement.find_element(By.TAG_NAME, "img")
    srcset = img_element.get_attribute("srcset")
    
    first_link = srcset.split(",")[0].strip().split(" ")[0]
    
    driver.quit()
    saveImage(first_link, dirName, fileName)
    return kernelImage(dirName, fileName)
    

In [40]:
# def readJson(pathname, filename):
#     data = pd.read_json("json/link/"+pathname+"/"+filename)
#     return data

In [16]:
def readJson(pathname, filename):
    data = pd.read_json("json/product/"+pathname+"/"+filename)
    return data

In [42]:
# df = readJson("Beds","Beds_datas.json")

In [43]:
# df = pd.json_normalize(df['Beds'])

In [44]:
# df.drop(columns="imgBase64", inplace=True)

In [45]:
# df = df[df['brandName'] != 'ใหม่'].reset_index(drop=True)


In [46]:

# pattern = r'\d{1,4}x\d{1,4}\s?ซม\.?'

# # ย้ายค่าที่ตรงกับ pattern จาก 'color' ไปยัง 'size'
# df.loc[df['size'].isna() & df['color'].str.contains(pattern, na=False), 'size'] = df['color'].str.extract(f'({pattern})').iloc[:, 0]

# # ลบค่าขนาดจากคอลัมน์ 'color'
# df['color'] = df['color'].str.replace(pattern, 'None', regex=True).str.strip()
# df['color']

In [47]:
# def saveProductDataToJson(dirName, fileName):
#     links = readJson(dirName, fileName)
#     datas = []
#     
#     try:
#         for i in links[0]:
#             brandName, productName,color, size, price = getDetailProduct(i)
#             stringDesc , codeProduct = getDescFromProduct(i)
#             imgBase64 = getImage(i, dirName, codeProduct)
#     
#             datas.append({
#                 "Code":codeProduct,
#                 "brandName":brandName,
#                 "productName":productName,
#                 "color":color,
#                 "size":size,
#                 "price":price,
#                 "imgBase64":imgBase64,
#                 "desc":stringDesc
#             })
#             
#         datas = {dirName:datas}
#         os.makedirs("json/product/" + dirName, exist_ok=True)
#         try:
#             with open(f'json/product/{dirName}/{dirName}_datas.json', 'w', encoding='utf-8') as FILE:
#                 json.dump(datas, FILE, ensure_ascii=False, indent=4)
#         except FileNotFoundError:
#             print(" Can not save data")
#             
#     except Exception as e:
#         print(e)

In [48]:
# saveProductDataToJson("dishwashing-accessories", "dishwashing-accessoriesLink.json")

1h 54m : 206 item

In [17]:
directory_path = 'json/link'

# สร้าง DataFrame ว่างเพื่อเก็บข้อมูลทั้งหมด
all_data = pd.DataFrame()

# ใช้ os.walk เพื่อเดินทางในไดเรกทอรี่ทั้งหมด
for root, dirs, files in os.walk(directory_path):
    for file in files:
        if file.endswith('.json'):
            # สร้าง path ของไฟล์ JSON
            file_path = os.path.join(root, file)

            # ดึงชื่อโฟลเดอร์
            folder_name = os.path.basename(root)

            # อ่านข้อมูลจากไฟล์ JSON
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)

                # แปลงข้อมูลเป็น DataFrame และเก็บข้อมูล folder_name
                df = pd.DataFrame(data, columns=['link'])
                df['folder_name'] = folder_name  # เพิ่มคอลัมน์ใหม่ที่เก็บชื่อโฟลเดอร์

                # นำ DataFrame ไปต่อรวมกับ all_data
                all_data = pd.concat([all_data, df], ignore_index=True)

In [18]:
def getB64ProductImg(uri):
    driver = openDriver()
    driver.get(uri)
    findElement = driver.find_element(By.XPATH, "//div[@class='pip-zoom-image']")
    img_element = findElement.find_element(By.TAG_NAME, "img")
    srcset = img_element.get_attribute("srcset")
    first_link = srcset.split(",")[0].strip().split(" ")[0]
    driver.quit()
    
    imgB64 = requests.get(first_link).content
    imgB64 = base64.b64encode(imgB64)
    imgB64 = imgB64.decode("utf-8")
    return imgB64

In [19]:
def getProductDataInSubImg(element, all_data = all_data):
    element = element.find_elements(By.CLASS_NAME, "pub__shoppable-image__dot")
    linkProduct = [i.get_attribute("href") for i in element]
    code = []
    
    for i in linkProduct:
        dataSet = []
        brandName, productName,color, size, price = getDetailProduct(i)
        stringDesc , codeProduct = getDescFromProduct(i)
        imgBase64 = getB64ProductImg(i)
        cate = ""
        matched_row = all_data[all_data['link'] == i]

        if not matched_row.empty:
            categoryName = matched_row['folder_name'].values[0] 
            dataSet.append({
                "cate": categoryName,
                "Code": codeProduct,
                "brandName": brandName,
                "productName": productName,
                "color": color,
                "size": size,
                "price": price,
                "imgBase64": imgBase64,
                "desc": stringDesc
            })
            
            os.makedirs("json/product/product-in-subImg", exist_ok=True)
            try:
                with open('json/product/product-in-subImg/product-in-subImg-data.json', 'r', encoding='utf-8') as FILE:
                    existingData = json.load(FILE)
    
            except FileNotFoundError:
                existingData = []
    
            existingData.extend(dataSet)
            with open('json/product/product-in-subImg/product-in-subImg-data.json', 'w', encoding='utf-8') as FILE:
                json.dump(existingData, FILE, ensure_ascii=False, indent=4)
            
        else:
            dataSet.append({
                "cate": cate,
                "Code":codeProduct,
                "brandName":brandName,
                "productName":productName,
                "color":color,
                "size":size,
                "price":price,
                "imgBase64":imgBase64,
                "desc":stringDesc
            })
            
            os.makedirs("json/product/no-category-data", exist_ok=True)
            try:
                with open('json/product/no-category-data/no-categot-product-data.json', 'r', encoding='utf-8') as FILE:
                    existingData = json.load(FILE)
            
            except FileNotFoundError:
                existingData = []
            
            existingData.extend(dataSet)
            with open('json/product/no-category-data/no-categot-product-data.json', 'w', encoding='utf-8') as FILE:
                json.dump(existingData, FILE, ensure_ascii=False, indent=4)
            
        code.append(codeProduct)
        
    return code

In [20]:
def getSubImgInRoom(driver, roomName):
    subImgInRoom = (driver.
                   find_elements(By.XPATH, 
                   "//div[@class='pub__shoppable-image pub__shoppable-image--visible-dots']")
                   )
    dataSet = []
    idSet = []
    j = 0
    os.makedirs("json/room", exist_ok=True)
    
    for i in subImgInRoom:
        
        subImg = i.find_element(By.TAG_NAME, "img").get_attribute("srcset").split(",")[0].strip().split(" ")[0]
        subImg = requests.get(subImg).content
        subImg = base64.b64encode(subImg)
        subImg = subImg.decode('utf-8')
        
        idSubImg = str(j) + roomName
        j = j + 1
        
        productCode = getProductDataInSubImg(i)
        
        dataSet.append({
            "id":idSubImg,
            "subImg":subImg,
            "productCode":productCode
        })
        
        idSet.append(idSubImg)
        
        
    try:
        with open('json/room/sub-img-rooms-data.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)
    
    except FileNotFoundError:
        existingData = []
    
    existingData.extend(dataSet)
    with open('json/room/sub-img-rooms-data.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
    
    return idSet
    

In [21]:
def saveDataInRoom(uri, roomName):
    browser = openDriver()
    browser.get(uri)
    body = browser.find_element(By.TAG_NAME, "body")
    
    for _ in range(10):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        
    titleDescRoom = browser.find_element(By.XPATH, "//h1[@class='c19ofv4i pub__h1 s1gshh7t']").text
    desc = browser.find_element(By.XPATH, "//div[@class='i1ycpxq9 m1e9rlhw pub__designSystemText t91kxqv w1fdzi2f']").text
    mainImage = browser.find_element(By.XPATH, "//span[@class='pub__aspect-ratio-box pub__aspect-ratio-box--wide']").find_element(By.TAG_NAME, "img").get_attribute("srcset").split(",")[0].strip().split(" ")[0]
    
    mainImage = requests.get(mainImage).content
    mainImage = base64.b64encode(mainImage)
    mainImage = mainImage.decode("utf-8") #base64 in form string
    
    idSubImg = getSubImgInRoom(browser, roomName)
    
    browser.quit()
    
    return titleDescRoom, desc, mainImage, idSubImg

    

In [22]:
def saveRoomData(uriRoom, roomName):
    titleDescRoom, desc, mainImage, idSubImg = saveDataInRoom(uriRoom, roomName)
    roomData = [
        {
            "roomName": roomName,
            "title":titleDescRoom,
            "desc":desc,
            "mainImage":mainImage,
            "idSubImg":idSubImg        
        }
    ]
    
    os.makedirs("json/room", exist_ok=True)
    try:
        with open('json/room/rooms-data.json', 'r', encoding='utf-8') as FILE:
            existingData = json.load(FILE)

    except FileNotFoundError:
        existingData = []

    existingData.extend(roomData)
    with open('json/room/rooms-data.json', 'w', encoding='utf-8') as FILE:
        json.dump(existingData, FILE, ensure_ascii=False, indent=4)
        

In [55]:
saveRoomData("https://www.ikea.com/th/th/rooms/living-room/gallery/a-retro-living-room-with-lots-of-storage-pubfe564a00", "living-room")

In [25]:
saveRoomData("https://www.ikea.com/th/th/rooms/dining/gallery/traditional-dining-room-that-satisfies-modern-needs-pub8eb11770","dining")

In [ ]:
saveRoomData("https://www.ikea.com/th/th/rooms/kitchen/gallery/a-modern-kitchen-for-a-classical-apartment-pubfe469c70","kitchen")

In [ ]:
saveRoomData("https://www.ikea.com/th/th/rooms/outdoor/gallery/a-generous-outdoor-living-room-to-enjoy-with-family-and-friends-pube08cd120","outdoor")

In [ ]:
saveRoomData("https://www.ikea.com/th/th/rooms/home-office/gallery/working-from-home-in-style-pub4e6d71af","home-office")

In [ ]:
saveRoomData("https://www.ikea.com/th/th/rooms/bathroom/gallery/perfect-organisation-with-a-classic-minimalist-look-puba0181bc0","bathroom")

In [23]:
saveRoomData("https://www.ikea.com/th/th/rooms/bedroom/gallery/a-colourful-bedroom-haven-for-nights-of-better-sleep-pub1285c0f0","bedroom")

In [104]:
df = pd.read_json("json/product/product-in-subImg/product-in-subImg-data.json")
df

,cate,Code,brandName,productName,color,size,price,imgBase64,desc
0,Beds,305.306.49,ÄLVDALEN แอลฟ์ดาเลน,โซฟาเบด 3 ที่นั่ง,คนีซา เบจ-เทา,None,"12,990",/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAEBAQEBAQECAg...,ผ้าคลุมนี้ทำจากผ้าโพลีเอสเตอร์ KNISA/คนิสา ที่...
1,Bed Textile,304.907.71,ÄNGSLILJA เอ็งส์ลิลยา,ปลอกผ้านวมและปลอกหมอน,สีเบจเทาอ่อน,150x200/50x80 ซม.,990,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAMDAwMDAwMEBA...,ปลอกผ้านวมผ่านการซักด้วยกระบวนการที่ทำให้ได้สี...
2,lamps,605.209.03,BLÅSVERK บลัวสแวร์ค,โคมแขวนเพดาน,เบจ,37 ซม.,"1,290",/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAUFBQUFBQYGBg...,หลอดไฟแยกจำหน่าย แนะนำให้ใช้หลอด LED หลอดกลมแก...
3,Bed Textile,105.430.11,SVARTPOPPEL สวาร์ทพ็อพเพิล,ปลอกหมอนอิง,เหลือง,50x50 ซม.,149,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAICAgICAgICAg...,ทอด้วยโพลีเอสเตอร์รีไซเคิล การนำของเสียมาใช้เป...
4,noticeboards,803.208.04,SKÅDIS สกัวดิส,แผ่นแขวนจัดเก็บ,ขาว,36x56 ซม.,499,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAUFBQUFBQYGBg...,ให้คุณออกแบบตู้เก็บของได้เอง เพียงประกอบแผ่นไม...
5,decorative-lighting,905.306.46,VATTENSTEN วัตเทนเสตียน,แถบไฟ LED,หลากสี,3 ม.,790,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAICAgICAgMDAw...,แสงไฟจากแถบไฟ LED สามารถมอบบรรยากาศที่อบอุ่นได...
6,paper-media-boxes,205.207.21,UPPDATERA อุปป์ดาเทียร่า,กล่องเก็บของ,ไม้ไผ่สีอ่อน,16x24x15 ซม.,399,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAEBAQEBAQECAg...,ทำจากไม้ไผ่ ซึ่งเป็นวัสดุธรรมชาติที่ทนทานและโต...
7,paper-media-boxes,904.681.64,PALLRA พัลร่า,ตู้ลิ้นชักเล็ก 3 ช่อง,ดำ,33x26 ซม.,590,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAEBAQEBAQECAg...,ปลายขาพลาสติกป้องกันรอยขีดข่วนบนพื้น และช่วยให...
8,Bed Textile,704.650.53,HANNELISE ฮันเนลีส,หมอนอิง,หลากสี,50x50 ซม.,399,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAEBAQEBAQEBAQ...,ปลอกหุ้มถอดซักเครื่องได้ จึงหมั่นดูแลรักษาให้เ...
9,bookcases shelving,405.305.78,EKET เอียคเกท,ตู้,น้ำตาล ลายไม้วอลนัท,35x35x35 ซม.,800,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAICAgICAgICAg...,มีรูเจาะสำหรับใส่เดือยไม้ จึงประกอบได้ง่ายและร...


In [107]:
df = pd.read_json("json/product/no-category-data/no-categot-product-data.json")
df

,cate,Code,brandName,productName,color,size,price,imgBase64,desc
0,,705.204.36,VÅGHÖJD โว้คเฮย,โคมไฟตั้งพื้น LED,หรี่ไฟได้ ดำ,None,"5,990",/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAICAgICAgICAg...,หลอดไฟ LED ไม่เพียงมีอายุการใช้งานยาวนานกว่าหล...
1,,204.509.97,PERSILLADE แพร์ชิลลาเด,กระถางต้นไม้,เทาเข้ม,24 ซม.,990,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAEBAQEBAQECAg...,ตกแต่งบ้านด้วยต้นไม้และกระถางที่สวยตรงสไตล์คุณ...
2,,105.355.44,HERRÅKRA แฮร์รัวครา,อาร์มแชร์,ดีเซอเริด เหลืองเข้ม,None,"3,590",/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAEBAQEBAQECAg...,ปลอกทำจากผ้าโพลีเอสเตอร์ Diseröd/ดีเซอเริด ที่...
3,,005.119.54,KONSTFULL คอนสต์ฟูล,แจกัน,น้ำตาล-เขียว,16 ซม.,590,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAMDAwMDAwQEBA...,แจกันออกแบบโดย Ilse Crawford โดยมีทั้งรูปร่างท...
4,,305.029.29,GRADVIS กราดวีส,กระถางต้นไม้,ใน/นอกอาคาร เทา,12 ซม.,199,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAQEBAQEBAUFBQ...,ตกแต่งบ้านด้วยต้นไม้และกระถางที่สวยตรงสไตล์คุณ...
5,,605.029.18,GRADVIS กราดวีส,แจกัน,เทา,19 ซม.,199,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAMDAwMDAwQEBA...,ใช้เป็นแจกันดอกไม้ หรือวางประดับห้อง เพื่อความ...
6,,204.509.97,PERSILLADE แพร์ชิลลาเด,กระถางต้นไม้,เทาเข้ม,24 ซม.,990,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAEBAQEBAQECAg...,ตกแต่งบ้านด้วยต้นไม้และกระถางที่สวยตรงสไตล์คุณ...
7,,603.542.77,MICKE มิคเก้,โต๊ะทำงาน,น้ำตาลดำ,142x50 ซม.,"3,990",/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAMDAwMDAwMDAw...,พร้อมตัวล็อกป้องกันลิ้นชักเลื่อนหลุดจากราง\n\n...
8,,005.047.36,OMBONAD อุมโบนาด,ถาด,ไม้วอลนัท,42 ซม.,"1,490",/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAICAgICAgICAg...,เหมาะสำหรับใช้เป็นถาดใส่อาหารเช้าหรือรองถ้วยชา...
9,,504.626.73,MARKFROST มาร์คฟรอสท์,โคมไฟตั้งโต๊ะ,หินอ่อน ดำ,None,590,/9j/4AAQSkZJRgABAQIAdgB2AAD/2wCEAAICAgICAgICAg...,โคมขนาดเล็ก จึงเคลื่อนย้ายสะดวก ให้แสงไฟช่วยสร...


In [140]:
dirName = {
    "bedroom":{
        "titleDescRoom" : nameDesc,
        "desc" : desc,
        "mainImageB64" : 1,
        "idImgSet" :{0,1,2}
    },
    "bathroom":{
        "titleDescRoom" : nameDesc,
        "desc" : desc,
        "mainImageB64" : 2,
        "idImgSet" :{0,1,2}
    }
}

ImgItemInRoom = [
    {   
        'id':0,
        "b64": 64,
        "itemCodeSet": {1, 2, 3, 4, 5}
    },
    {
        'id':1,
        "b64": 64,
        "itemCodeSet": {1, 2, 3, 4, 5}
    },
    {
        'id':2,
        "b64": 64,
        "itemCodeSet": {1, 2, 3, 4, 5}
    }
]

dirItemInRoom = {
    "cateName":[
        {
            "codeId" : 1,
            "brandName": "app",
            "productName": "lamp",
            "color": "หรี่ไฟได้ ขาว",
            "size": "80 ซม.",
            "price": "790บาท/ช",
            "imageB64" : "b64",
            "desc":"hello world"
        },
        {
            "codeId" : 2,
            "brandName": "app",
            "productName": "lamp",
            "color": "หรี่ไฟได้ ขาว",
            "size": "80 ซม.",
            "price": "790บาท/ช",
            "imageB64" : "b64",
            "desc":"hello world"
        }
    ]
}

---------